**Objective :**
> Create the model so as to predict the 'Drug Allergic'.



**Steps :**
1. Review data by checking the 'null value' in data. (In fact, I reviewed dataset on Microsoft Excel so as to see overall data first.)
2. Cleaning data.
3. Check correlation number for each feature with 'Drug_Rechallenge_Result' feature.
4. Select the algorithm that suitable for data & question.
5. Split dataset for training&testing.
6. Create a model.
7. Check the accuracy of model.
8. Improve model.
9. Compare result before and after improve the model.
10. Summarize.

---

**Let start!!**


First of all, I need to connect colab with google drive for make sure I can load data.

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load data set.

In [ ]:
import numpy as np
import pandas as pd

path='/content/drive/My Drive/Problem1/DrugAllergyKaggle_v3_050720.csv'
data = pd.read_csv(path)
# check size #
data.shape

(799, 18)

Check null value in dataset.

In [ ]:
# check null data
data.isnull().sum()

Patient_ID                   0
Gender                       0
Age_Year                     0
ELISpot_Control              0
ELISpot_Result               1
Naranjo_Score              368
Naranjo_Category           290
Exposure_Time              177
Steroid_Usage                0
Underlying_Condition_A      18
Underlying_Condition_B       0
Underlying_Condition_C       0
Underlying_Condition_D      14
Underlying_Condition_E       1
Suspicion_Score              0
Allergic_Reaction_Group     28
Drug_Group                   0
Drug_Rechallenge_Result    685
dtype: int64

Select ground truth data records that there are not null value in column 'Drug_Rechallenger_Result'.

In [ ]:
# Select ground truth data by compare is not null in colume 'Drug_Rechallenger_Result'.
data_ground_truth = data.loc[(data['Drug_Rechallenge_Result'].notnull())]
# Re-Index
data_ground_truth = data_ground_truth.reset_index(drop=True)
data_ground_truth.shape
data_ground_truth.head()

,Patient_ID,Gender,Age_Year,ELISpot_Control,ELISpot_Result,Naranjo_Score,Naranjo_Category,Exposure_Time,Steroid_Usage,Underlying_Condition_A,Underlying_Condition_B,Underlying_Condition_C,Underlying_Condition_D,Underlying_Condition_E,Suspicion_Score,Allergic_Reaction_Group,Drug_Group,Drug_Rechallenge_Result
0,79,1,55,2344,129.0,2.0,1.0,25.0,0,0.0,0,0,0.0,0.0,2,6.0,1,0.0
1,79,1,55,2344,113.0,3.0,1.0,25.0,0,0.0,0,0,0.0,0.0,2,6.0,3,1.0
2,188,1,47,2376,83.0,5.0,2.0,2.0,0,0.0,0,0,0.0,0.0,2,2.0,6,1.0
3,95,0,55,1624,63.0,NaN,2.0,12.0,0,0.0,0,1,0.0,1.0,2,1.0,7,0.0
4,7,0,75,1868,51.0,5.0,2.0,1.0,0,0.0,0,0,0.0,0.0,2,6.0,4,1.0


Check null data again.

In [ ]:
# check null data
data_ground_truth.isnull().sum()

Patient_ID                  0
Gender                      0
Age_Year                    0
ELISpot_Control             0
ELISpot_Result              0
Naranjo_Score              21
Naranjo_Category           14
Exposure_Time              21
Steroid_Usage               0
Underlying_Condition_A      0
Underlying_Condition_B      0
Underlying_Condition_C      0
Underlying_Condition_D      1
Underlying_Condition_E      0
Suspicion_Score             0
Allergic_Reaction_Group     0
Drug_Group                  0
Drug_Rechallenge_Result     0
dtype: int64

Cleaning data
Replace 'nan' value with 'most frequent' data by SimpleImputer function.

In [ ]:
from sklearn.impute import SimpleImputer

Naranjo_Score = data_ground_truth['Naranjo_Score'].values
Naranjo_Score = np.reshape(Naranjo_Score,(-1,1))

Naranjo_Category = data_ground_truth['Naranjo_Category'].values
Naranjo_Category = np.reshape(Naranjo_Category,(-1,1))

Exposure_Time = data_ground_truth['Exposure_Time'].values
Exposure_Time = np.reshape(Exposure_Time,(-1,1))

# Replacing 'nan' value with 'most frequent' data.
imp = SimpleImputer(missing_values = np.nan , strategy='most_frequent')
imp.fit(Naranjo_Score)
imp.fit(Naranjo_Category)
imp.fit(Exposure_Time)

data_ground_truth['Naranjo_Score'] = imp.transform(Naranjo_Score)
data_ground_truth['Naranjo_Category'] = imp.transform(Naranjo_Category)
data_ground_truth['Exposure_Time'] = imp.transform(Exposure_Time)

# check null data
data_ground_truth.isnull().sum()

Patient_ID                 0
Gender                     0
Age_Year                   0
ELISpot_Control            0
ELISpot_Result             0
Naranjo_Score              0
Naranjo_Category           0
Exposure_Time              0
Steroid_Usage              0
Underlying_Condition_A     0
Underlying_Condition_B     0
Underlying_Condition_C     0
Underlying_Condition_D     1
Underlying_Condition_E     0
Suspicion_Score            0
Allergic_Reaction_Group    0
Drug_Group                 0
Drug_Rechallenge_Result    0
dtype: int64

Check correlated number with 'Drug_Rechallenge_Result'
I choose features(column) that get the number of corr is close to 1.

In [ ]:
data_ground_truth.corr()['Drug_Rechallenge_Result']

Patient_ID                -0.107253
Gender                     0.053926
Age_Year                  -0.192368
ELISpot_Control            0.065338
ELISpot_Result             0.257566
Naranjo_Score              0.434784
Naranjo_Category           0.398901
Exposure_Time             -0.076193
Steroid_Usage             -0.073921
Underlying_Condition_A     0.082576
Underlying_Condition_B     0.069898
Underlying_Condition_C     0.019038
Underlying_Condition_D    -0.108515
Underlying_Condition_E    -0.004730
Suspicion_Score           -0.062073
Allergic_Reaction_Group   -0.155619
Drug_Group                 0.027581
Drug_Rechallenge_Result    1.000000
Name: Drug_Rechallenge_Result, dtype: float64

So the top 3 feature that get high correlation number are 
'Naranjo_Score', 'Naranjo_Category' and 'ELISpot_Result' 
and I decide to use all of them in my model.

In [ ]:
# So we choose 'ELISpot_Result', 'Naranjo_Score' and ''Naranjo_Category' features.
x1 = data_ground_truth[['ELISpot_Result', 'Naranjo_Score', 'Naranjo_Category']]
x1.head()

,ELISpot_Result,Naranjo_Score,Naranjo_Category
0,129.0,2.0,1.0
1,113.0,3.0,1.0
2,83.0,5.0,2.0
3,63.0,0.0,2.0
4,51.0,5.0,2.0


In [ ]:
y1 = data_ground_truth['Drug_Rechallenge_Result']
y1.head()

0    0.0
1    1.0
2    1.0
3    0.0
4    1.0
Name: Drug_Rechallenge_Result, dtype: float64

The best algorithm for prediction 'YES'/'NO' answer, I decide to choose ''Logistic Regression' algorithm.

In [ ]:
from sklearn.linear_model import LogisticRegression
# Split data to train & test so as to protect 'over/under fitting'
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.3, random_state=42)

# Create a 'Logistic Regression' Model
LGT_Model = LogisticRegression(solver='lbfgs')
# Training Model
LGT_Model.fit(x1_train, y1_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Check the accuracy of model.

In [ ]:
y1_pred = LGT_Model.predict(x1_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('ACC ',accuracy_score(y1_test, y1_pred))
print(classification_report(y1_test, y1_pred))

ACC  0.7714285714285715
              precision    recall  f1-score   support

         0.0       0.77      0.96      0.85        24
         1.0       0.80      0.36      0.50        11

    accuracy                           0.77        35
   macro avg       0.78      0.66      0.68        35
weighted avg       0.78      0.77      0.74        35



Now, The accuracy of this model is 77%, this number is quite high.

In [ ]:
# Let try to predict
# LGT_Model.predict([[A,B,C]])
# - A is ELISpot_Result
# - B is Naranjo_Score 
# - C is Naranjo_Category

print(LGT_Model.predict([[113,3,1]]))
print(LGT_Model.predict_proba([[113,3,1]]))

[1.]
[[0.00336571 0.99663429]]


Improve the model
1. Implement the StandardScaler into training data (variable x).
2. Increase the % of training data from 70% to 80%.

In [ ]:
# Let check variance before scale.
print(x1.var())

ELISpot_Result      2198.087952
Naranjo_Score          3.668374
Naranjo_Category       0.318584
dtype: float64


In [ ]:
x2 = data_ground_truth[['ELISpot_Result', 'Naranjo_Score', 'Naranjo_Category']]
y2 = data_ground_truth['Drug_Rechallenge_Result']

# Implement the StandardScaler
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
x2 = pd.DataFrame(scale.fit_transform(x2), columns=x2.columns)

In [ ]:
# Let check variance after scale.
print(x2.var())

ELISpot_Result      1.00885
Naranjo_Score       1.00885
Naranjo_Category    1.00885
dtype: float64


In [ ]:
# Split the data to training & testing again and increase the % of training from 70% to 80%.
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=42)

LGT_Model2 = LogisticRegression(solver='lbfgs')
LGT_Model2.fit(x2_train, y2_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y2_pred = LGT_Model2.predict(x2_test)

print('ACC ',accuracy_score(y2_test, y2_pred))
print(classification_report(y2_test, y2_pred))

ACC  0.8260869565217391
              precision    recall  f1-score   support

         0.0       0.83      0.94      0.88        16
         1.0       0.80      0.57      0.67         7

    accuracy                           0.83        23
   macro avg       0.82      0.75      0.77        23
weighted avg       0.82      0.83      0.82        23



Now, The accuracy of this model is increase from 77% to 83%.

Compare with Model1

In [ ]:
y1_pred = LGT_Model.predict(x1_test)
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('ACC ',accuracy_score(y1_test, y1_pred))
print(classification_report(y1_test, y1_pred))

ACC  0.7714285714285715
              precision    recall  f1-score   support

         0.0       0.77      0.96      0.85        24
         1.0       0.80      0.36      0.50        11

    accuracy                           0.77        35
   macro avg       0.78      0.66      0.68        35
weighted avg       0.78      0.77      0.74        35



Summarrize
From the classification report, the accuracy number is increase from 77% to 83% after I increase and implement the standard scaler into training dataset. And the number of precision, recall and f1-score are increase too.

So the main impact of accuracy is the training dataset, if we have more groud truth data, this model will have more accurate.